### Q1. 항암약 위약을 투여 받은 환자의 부작용은 감기약 위약을 투여 받은 환자의 부작용 분포와 차이가 있는가?

항암약(위약)을 투여 받은 환자 그룹의 관찰된 부작용

In [1]:
import pandas as pd
df = pd.DataFrame({ "항암약":[4,4,3,4,1,4,1,4,1,4,4,2,1,4,2,3,2,4,4,4] })
# 1: '아픔', 2: '조금 아픔', 3: '속쓰림', 4: '무증상'

감기약(위약)을 투여받은 환자들 중의 알려진 부작용 발생 비율


In [2]:
# 1. 아픔: 10%
# 2. 조금 아픔 5%
# 3. 속 쓰림 15%
# 4. 무증상: 70%
df.head()

,항암약
0,4
1,4
2,3
3,4
4,1


-	귀무가설(H0): 감기약(위약)의 부작용과 항암약(위약)의 부작용은 동일하다.
- 	대립가설(H1): 감기약(위약)의 부작용과 항암약(위약)의 부작용은 다르다.

 ### 1-1. 항암약을 투여 받은 환자 중 '무증상'의 비율을 0과 1사이로 구하시오.

In [10]:
cond1 = df['항암약'] == 4 #무증상 환자인 경우
percentage = (len(df[cond1]) / len(df))
print(percentage)

0.55


In [9]:
# 또 다른 방법
print(df['항암약'].value_counts(normalize = True)) #value_counts 사용 시 각 항목별 총 수가 나오고, normalize 사용시 항목의 비율이 출력된다.

항암약
4    0.55
1    0.20
2    0.15
3    0.10
Name: proportion, dtype: float64


항암약
4    0.55
1    0.20
2    0.15
3    0.10
Name: proportion, dtype: float64

### 1-2. 감기약의 예상 부작용 비율과 항암약의 부작용 관찰값이 통계적으로 유의미하게 차이가 있는지 확인하려 한다. 카이 제곱 검정을 사용하여 검정 통계량을 구하시오.

In [24]:
# ob(관찰값)와 ex(기대값)를 구해야 함. ob는 작은 수부터 정렬.
# ex는 전체 데이터 수(20)에 각 증상에 대한 비율을 곱해 줌.

ob = df['항암약'].value_counts().sort_index().to_list() # 관찰값. 빈도수를 오름차순 정렬 후 리스트화
# 잘 모르겠는 경우에는 직접 적어도 무방
# ob  = [4,3,2,11] 1/2/3/4의 관찰 빈도수
ex = [0.1 * 20, 0.05 * 20, 0.15 * 20, 0.7 * 20] # 기대값 (10% / 5%/ 15%/ 7%)
print(ob)
print(ex)

# 관찰빈도수와 기대빈도수를 구했으면 카이제곱검정을 시행.
# 카이제곱검정 함수는 관찰값(리스트) / 기대값(리스트) 순서로 값을 넣는다.
from scipy import stats

# print(dir(stats.chisquare)/)
#print(help(stats.chisquare))
result = stats.chisquare(ob, ex)
print(result.statistic) #검정통계량 - 6.976190476190476


[4, 3, 2, 11]
[2.0, 1.0, 3.0, 14.0]
6.976190476190476


### 1-3. 위의 p-값을 구하시오

In [25]:
print(result.pvalue) #0.07266054733847573
# 약 0.07. 신뢰구간 5%를 초과하므로 귀무가설을 채택. 감기약과 항암제의 부작용 확률은 동일함.

0.07266054733847573


### Q2. 다중 선형 회귀 문제

In [27]:
import pandas as pd
df = pd.read_csv("data6-3-2.csv")
df.head(2)

,solar,wind,o3,temperature
0,89.14,6.28,33.52,23.0
1,109.97,1.04,27.01,20.7


1. 다중 선형 회귀 모델을 구축하고, 독립변수 o3의 회귀계수를 구하시오.
- 독립변수: solar(태양 에너지), wind(바람의 세기), o3(오존 농도)
- 종속변수: temperature(온도)


In [30]:
from statsmodels.formula.api import ols
formula = 'temperature ~ solar + wind + o3' #종속변수 우선, 물결 후 독립변수들을 넣어준다. 여러개의 독립변수는 +로 잇는다.
model = ols(formula, data=df).fit()
print(model.summary())

#회귀계수는 coef -> 변수별로 확인 가능.
print(model.params['o3'])

                            OLS Regression Results                            
Dep. Variable:            temperature   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     1.464
Date:                Sat, 15 Jun 2024   Prob (F-statistic):              0.229
Time:                        08:53:43   Log-Likelihood:                -195.45
No. Observations:                 100   AIC:                             398.9
Df Residuals:                      96   BIC:                             409.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     19.0507      1.994      9.555      0.0

2. 데이터에서 'solar'와 'o3' 값을 고정한 상태에서, 'wind'의 세기가 증가함에 따라 'temperature'가 감소하는지를 검증하기 위해 다중 선형 회귀 분석을 수행하고, 이 때'wind'의 회귀 계수에 대한 p-value 값을 구하시오. (유의수준: 0.05)

In [33]:
model.pvalues['wind']
# 약 0.78. 감소하지 않는다.

0.7797177202071661

3. solar:100, wind:5, o3:30일 때 예측값과 그에 대한 95% 신뢰구간을 구하시오.

In [40]:
new_data = pd.DataFrame({
    'solar':[100],
    'wind':[5],
    'o3':[30]
})

# 값을 하나 넣더라도 반드시 리스트형으로 감싸자. 그렇지 않으면 만들어지지 않음!

pred = model.get_prediction(new_data)
print(pred.summary_frame(alpha = 0.05)) # summary가 아닌 summary_frame으로 적어야 함. 유의구간은 alpha로 작성

# 예측값은 21.56163
# 신뢰구간은 21.213737 ~ 21.909524


       mean   mean_se  mean_ci_lower  mean_ci_upper  obs_ci_lower  \
0  21.56163  0.175263      21.213737      21.909524     18.082985   

   obs_ci_upper  
0     25.040276  
